In [1]:
from amuse.units import units
from amuse.lab import Huayno, nbody_system, new_galactics_model
from amuse.lab import Gadget2
from amuse.community.ph4.interface import ph4
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from amuse.couple import bridge
from amuse.community.hermite.interface import Hermite
from amuse.community.bhtree.interface import BHTree
from astropy.io import fits

In [2]:
%%time

# Milky Way := Mdisk = 4.5 Mbulge | Mhalo = 100 Mbulge
# Andromeda := Mdisk = 4-7 Mbulge | Mhalo = 87 Mbulge
# Mdisk = 5 Mbulge, Mhalo = 95 Mbulge

n_halo  = 20000
n_bulge = 10000
n_disk  = 10000
M_galaxy = 1e12 | units.MSun
R_galaxy = 80  | units.kpc
converter = nbody_system.nbody_to_si(M_galaxy, R_galaxy)

galaxy1 = new_galactics_model(n_halo,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge,
                                disk_number_of_particles=n_disk)
galaxy1.move_to_center()
# Order of particles: Disk => Bulge => Halo

CPU times: user 14.8 s, sys: 2.13 s, total: 16.9 s
Wall time: 19.1 s


In [3]:
%%time
n_halo_test  = 2000
n_disk_test  = 1000
n_bulge_test = 1000

test_particles_1 = new_galactics_model(n_halo_test,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge_test,
                                disk_number_of_particles=n_disk_test)

test_particles_1.move_to_center()
test_particles_1.mass = 0 |units.kg

test_particles_2 = new_galactics_model(n_halo_test+1,
                                converter,
                                do_scale=True,
                                bulge_number_of_particles=n_bulge_test+1,
                                disk_number_of_particles=n_disk_test+1)

test_particles_2.move_to_center()
test_particles_2.mass = 0 |units.kg

CPU times: user 3.96 s, sys: 2.37 s, total: 6.33 s
Wall time: 7.32 s


In [4]:
print(galaxy1)

                 key         mass           vx           vy           vz            x            y            z
                   -           kg    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========
10875095843854742828    4.304e+36   -1.873e+05   -4.310e+03    2.116e+03   -2.452e+19    2.955e+20    1.138e+19
 3452269457190091868    4.304e+36    1.747e+05   -6.661e+04    6.072e+03   -2.875e+19   -4.857e+19    4.396e+18
13751367681017400641    4.304e+36   -1.209e+05   -1.370e+05   -8.543e+03   -1.669e+20    1.534e+20   -1.032e+19
 2592027311382297722    4.304e+36    1.719e+05    5.996e+04   -1.010e+04    8.508e+19   -3.362e+20   -3.121e+17
13076103506856783916    4.304e+36    1.952e+05    3.951e+03    8.492e+03    5.888e+17   -3.819e+20   -1.150e+19
14105269068257225326    4.304e+36   -1.911e+05   -1.801e+04    3.971e+03   -3.061e+19    2.633e+20   -4.

In [5]:
print(test_particles_1)

                 key         mass           vx           vy           vz            x            y            z
                   -           kg    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========
 2912352189413867787    0.000e+00   -1.878e+05   -8.034e+03    2.056e+03   -2.589e+19    2.946e+20    1.133e+19
12900895475112855233    0.000e+00    1.746e+05   -7.040e+04    6.016e+03   -3.008e+19   -4.540e+19    4.428e+18
13196190655374549951    0.000e+00   -1.213e+05   -1.408e+05   -8.614e+03   -1.666e+20    1.541e+20   -1.011e+19
 7050974111530681869    0.000e+00    1.719e+05    5.630e+04   -1.018e+04    8.241e+19   -3.296e+20   -2.236e+17
 4673846032669660385    0.000e+00    1.951e+05    2.349e+02    8.438e+03   -1.082e+18   -3.748e+20   -1.128e+19
 2607727313782020651    0.000e+00   -1.915e+05   -2.175e+04    3.912e+03   -3.191e+19    2.627e+20   -4.

In [6]:
print(test_particles_2)

                 key         mass           vx           vy           vz            x            y            z
                   -           kg    m * s**-1    m * s**-1    m * s**-1            m            m            m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========
 5569415534887972413    0.000e+00   -1.879e+05   -8.071e+03    2.049e+03   -2.597e+19    2.952e+20    1.135e+19
 1176093441841309330    0.000e+00    1.745e+05   -7.043e+04    6.009e+03   -3.017e+19   -4.544e+19    4.440e+18
 7247257094103620223    0.000e+00   -1.214e+05   -1.409e+05   -8.621e+03   -1.669e+20    1.544e+20   -1.013e+19
10419877632957027452    0.000e+00    1.717e+05    5.626e+04   -1.018e+04    8.251e+19   -3.301e+20   -2.201e+17
13586702358757339627    0.000e+00    1.950e+05    1.979e+02    8.431e+03   -1.122e+18   -3.754e+20   -1.129e+19
 8591295143880942043    0.000e+00   -1.917e+05   -2.178e+04    3.905e+03   -3.200e+19    2.632e+20   -4.

In [7]:
for i in range(10):
    print((test_particles_1[i].x - test_particles_2[i].x).in_(units.parsec))

2.62342287414 parsec
2.84948725324 parsec
10.226620306 parsec
-3.22824509659 parsec
1.28279406188 parsec
2.94855429945 parsec
0.193812783865 parsec
-3.94763796256 parsec
-9.75030160359 parsec
1.8510242535 parsec


In [8]:
for i in range(10):
    print((galaxy1[i].x - test_particles_2[i].x).in_(units.parsec))

47.1660916741 parsec
45.7701619493 parsec
0.213417991313 parsec
83.300057548 parsec
55.4443874244 parsec
45.1584307464 parsec
62.1687425045 parsec
87.7422518002 parsec
123.569336406 parsec
51.9356292625 parsec


In [9]:
for i in range(10):
    print((test_particles_1[i].vx - test_particles_2[i].vx).in_(units.kms))

0.131864038512 kms
0.140838973809 kms
0.133502077358 kms
0.14077103954 kms
0.141347708184 kms
0.131771259627 kms
0.132758981341 kms
0.132321880631 kms
0.135504582302 kms
0.1315577153 kms


In [10]:
%%time
disk_test1 = test_particles_1[:1000]
dist_test2 = test_particles_2[:1000]




for i in range(10):
    distance = (((disk_test1[:].x - disk_test1[i].x)**2 + (disk_test1[:].y - disk_test1[i].y)**2 +(disk_test1[:].z - disk_test1[i].z)**2)**0.5).in_(units.parsec)
    distance_to_rest = np.delete(distance,i)
    print(np.mean(distance_to_rest),distance_to_rest.min())

11650.3978821 3.08567758128e+16 * m 1168.41101772 parsec
7300.18127356 3.08567758128e+16 * m 385.818479387 parsec
10022.2487389 3.08567758128e+16 * m 269.696617974 parsec
12860.6131376 3.08567758128e+16 * m 477.009747431 parsec
13812.233069 3.08567758128e+16 * m 622.649604059 parsec
10866.8653563 3.08567758128e+16 * m 706.578573325 parsec
7543.57177743 3.08567758128e+16 * m 288.386016649 parsec
9197.93434884 3.08567758128e+16 * m 675.339934246 parsec
9528.70742995 3.08567758128e+16 * m 795.97906519 parsec
7466.270959 3.08567758128e+16 * m 305.462671146 parsec
CPU times: user 163 ms, sys: 1.66 ms, total: 165 ms
Wall time: 162 ms


In [11]:
bulge_test1 = test_particles_1[1000:2000]
bulge_test2 = test_particles_2[1000:2000]




for i in range(10):
    distance = (((bulge_test1[:].x - bulge_test1[i].x)**2 + (bulge_test1[:].y - bulge_test1[i].y)**2 +(bulge_test1[:].z - bulge_test1[i].z)**2)**0.5).in_(units.parsec)
    distance_to_rest = np.delete(distance,i)
    print(np.mean(distance_to_rest),distance_to_rest.min())

1754.24603145 3.08567758128e+16 * m 194.385095449 parsec
1475.24071105 3.08567758128e+16 * m 180.974779239 parsec
3340.15932968 3.08567758128e+16 * m 321.940399563 parsec
1872.02841576 3.08567758128e+16 * m 218.5917197 parsec
2125.43520972 3.08567758128e+16 * m 63.176860519 parsec
2135.19999359 3.08567758128e+16 * m 219.048422464 parsec
1426.22892351 3.08567758128e+16 * m 160.359822155 parsec
1396.36915762 3.08567758128e+16 * m 37.0468209748 parsec
2069.46439529 3.08567758128e+16 * m 336.655470134 parsec
3322.85119148 3.08567758128e+16 * m 104.622675948 parsec


In [12]:
# add a random position to the test_particles
np.random.seed(94)

random_x = (np.random.random(n_bulge_test+n_disk_test) * 200) - 100 |units.parsec
random_y = (np.random.random(n_bulge_test+n_disk_test) * 200) - 100 |units.parsec
random_z = (np.random.random(n_bulge_test+n_disk_test) * 200) - 100 |units.parsec

print(random_x[:5])
print(random_y[:5])
print(random_z[:5])

test_particles_1.x[:n_bulge_test+n_disk_test] += random_x
test_particles_1.y[:n_bulge_test+n_disk_test] += random_y
test_particles_1.z[:n_bulge_test+n_disk_test] += random_z

[43.8771572488, 21.4442351244, 37.9723229158, -15.3134042199, -9.84677788195] parsec
[58.7392781174, -52.5265654689, -62.1677439811, -41.6142238762, 64.3661956002] parsec
[-18.1025436799, -35.5169512141, 65.3464247544, 27.1906112106, -45.3286916296] parsec


In [13]:
converter = nbody_system.nbody_to_si(1e12|units.MSun, 100|units.kpc)
dynamics = BHTree(converter,number_of_workers = 1)
dynamics.parameters.epsilon_squared = (100|units.parsec)**2
dynamics.parameters.timestep = 1 |units.Myr
set1 = dynamics.particles.add_particles(galaxy1)

In [14]:
test_particles_1_stars = test_particles_1[:int(n_bulge_test+n_disk_test)]
star_dynamics = BHTree(converter)
star_set_1 = star_dynamics.particles.add_particles(test_particles_1_stars)
star_dynamics.parameters.timestep = 1|units.Myr

In [15]:
gravity = bridge.Bridge(use_threading=False)
gravity.add_system(star_dynamics, (dynamics,) )
gravity.add_system(dynamics)
gravity.timestep = 1|units.Myr
channel_stars = star_dynamics.particles.new_channel_to(star_dynamics.particles)
channel_galaxies = dynamics.particles.new_channel_to(dynamics.particles)

In [17]:
#channel = dynamics.particles.new_channel_to(dynamics.particles)
times = np.arange(0., 101, 1) | units.Myr
threshold = 10. |units.Myr 
for time in tqdm(range(len(times))):
    gravity.evolve_model(times[time])
    if times[time] %threshold == 0|units.Myr:
        channel_galaxies.copy()
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_disk)].value_in(units.kpc), set1.y[:int(n_bulge)].value_in(units.kpc), s=0.3,label = 'Disk')
        plt.scatter(set1.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.y[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Bulge')
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-25,25)
        #plt.axis("equal")
        plt.ylim(-25,25)
        plt.savefig("test_plots_xy/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        plt.figure(figsize = (10,8))
        plt.scatter(set1.x[:int(n_disk)].value_in(units.kpc), set1.z[:int(n_bulge)].value_in(units.kpc), s=0.3,label = 'Disk')
        plt.scatter(set1.x[int(n_disk):int(n_disk+n_bulge)].value_in(units.kpc), set1.z[int(n_disk):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Bulge')
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("z [kpc]")
        plt.xlim(-25,25)
        #plt.axis("equal")
        plt.ylim(-25,25)
        plt.savefig("test_plots_xz/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        channel_stars.copy()
        plt.figure(figsize = (10,8))
        plt.scatter(star_set_1.x[:int(n_disk_test)].value_in(units.kpc), star_set_1.y[:int(n_bulge_test)].value_in(units.kpc), s=0.3,label = 'Disk')
        plt.scatter(star_set_1.x[int(n_disk_test):int(n_disk_test+n_bulge_test)].value_in(units.kpc), star_set_1.y[int(n_disk_test):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Bulge')
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("y [kpc]")
        plt.xlim(-25,25)
        #plt.axis("equal")
        plt.ylim(-25,25)
        plt.savefig("test_plots_stars_xy/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
        plt.figure(figsize = (10,8))
        plt.scatter(star_set_1.x[:int(n_disk_test)].value_in(units.kpc), star_set_1.z[:int(n_bulge_test)].value_in(units.kpc), s=0.3,label = 'Disk')
        plt.scatter(star_set_1.x[int(n_disk_test):int(n_disk_test+n_bulge_test)].value_in(units.kpc), star_set_1.z[int(n_disk_test):int(n_bulge+n_disk)].value_in(units.kpc), s=0.3,label = 'Bulge')
        plt.title("Disk + Bulge \n time = " +str(times[time]) )
        plt.legend()
        plt.xlabel("x [kpc]")
        plt.ylabel("z [kpc]")
        plt.xlim(-25,25)
        #plt.axis("equal")
        plt.ylim(-25,25)
        plt.savefig("test_plots_stars_xz/snap%04d.png"%time)
        #plt.show()
        plt.close()
        
gravity.stop()

100%|██████████| 101/101 [05:48<00:00,  3.45s/it]
